Prepairing Dataset for Model

In [ ]:
BATCH_SIZE = 64
def get_training_dataset(dataset):
    dataset = dataset.map(read_image_tfds, num_parallel_calls=16)
    dataset = dataset.shuffle(512, reshuffle_each_iteration=True)
    dataset = dataset.repeat()
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(-1)
    
    return dataset
def get_validation_dataset(dataset):
    dataset = dataset.map(read_image_tfds, num_parallel_calls = 16)
    dataset = dataset.shuffle(512, reshuffle_each_iteration=True)
    dataset = dataset.repeat()
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(-1)
    
    return dataset

Defining the Network Used

Here we are using MobileNetV2 Network

In [ ]:
def feature_extractor(inputs):
    mobilenet_model = tf.keras.applications.MobileNetV2(
        input_shape=(224,224,3),
        alpha=1.0,
        include_top= False,
        weights= None
    )
    feature_extractor = mobilenet_model(inputs)
    
    return feature_extractor

In [ ]:
def dense_layers(inputs):
    
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    
    return x

In [ ]:
def bounding_box_regression(x):
    bounding_box_regression_output = tf.keras.layers.Dense(4, activation='linear', name='bounding_box')(x)
    return bounding_box_regression_output

In [ ]:
def final_model(inputs):
    feature_cnn = feature_extractor(inputs)
    last_dense_layer = dense_layers(feature_cnn)
    bounding_box_output = bounding_box_regression(last_dense_layer)
    model = tf.keras.Model(inputs = inputs, outputs = bounding_box_output)
    
    return model

In [ ]:
def define_and_compile_model():
    inputs = tf.keras.layers.Input(shape=(224,224,3))
    model = final_model(inputs)
    
    model.compile(optimizer = tf.keras.optimizers.SGD(momentum=0.9),
                  loss = 'mse',
                  metrics='acc')
    return model

In [ ]:
model = define_and_compile_model()
model.summary()